Copyright 2022 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");

In [15]:
# @title License
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Action-Angle Networks



In [16]:
%pwd

'/Users/ameyad/Documents/google-research/action_angle_networks'

In [17]:
%mkdir -p ../notebook_outputs

In [18]:
# @title Base Imports
from typing import *
import functools
import sys
import tempfile
import os

from absl import logging
import collections
import chex
from clu import checkpoint
import numpy as np
import jax
import jax.numpy as jnp
import flax.linen as nn
from flax.core import frozen_dict
from flax.training import train_state
import optax
import distrax
import tensorflow as tf
import ml_collections
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib
import pysr
import yaml

PLT_STYLE_CONTEXT = ['science', 'ieee', 'grid']

sys.path.append("..")
matplotlib.rc("animation", html="jshtml")
logging.get_absl_handler().python_handler.stream = sys.stdout
logging.set_verbosity(logging.INFO)

In [19]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
# @title Source Imports
%autoreload 2
import harmonic_motion_simulation
import models
import train
import analysis
from configs.harmonic_motion import (action_angle_flow, action_angle_mlp, euler_update_flow, euler_update_mlp)

## Loading a Pre-Trained Model

In [23]:
# @title Location of Pretrained Model
# config_name = "euler_update_flow"
config_name = "action_angle_flow"
k_pair = "0.5"
num_samples = "500"
workdir = f"/Users/ameyad/Documents/google-research/workdirs/performance_vs_samples/action_angle_networks/configs/harmonic_motion/{config_name}/k_pair={k_pair}/num_samples={num_samples}"

In [24]:
config, scaler, state, aux = analysis.load_from_workdir(workdir)

INFO:absl:Saved config found. Loading...
INFO:absl:Using config: activation: sigmoid
batch_size: 100
encoder_decoder_type: flow
eval_cadence: 50
flow_type: shear
latent_size: 100
learning_rate: 0.001
model: action-angle-network
num_flow_layers: 20
num_samples: 1000
num_train_steps: 50000
num_trajectories: 2
polar_action_angles: true
regularizations:
  actions: 1.0
  angular_velocities: 0.0
  encoded_decoded_differences: 0.0
rng_seed: 0
scaler: standard
simulation: harmonic
simulation_parameter_ranges:
  A: !!python/tuple
  - 1
  - 10
  k_pair: !!python/tuple
  - 0.5
  k_wall: !!python/tuple
  - 0.005
  - 0.01
  m: !!python/tuple
  - 1
  - 5
  phi: !!python/tuple
  - 0
  - 1
single_step_predictions: true
split_on: times
test_split_proportion: 0.5
test_time_jumps: !!python/tuple
- 1
- 2
- 5
- 10
- 20
- 50
time_delta: 1.0
train_split_proportion: 0.5
train_time_jump_range: !!python/tuple
- 1
- 10
train_time_jump_schedule: linear

INFO:absl:Restoring checkpoint: /Users/ameyad/Documents/goog

In [27]:
train_positions = aux["train"]["positions"]
train_momentums = aux["train"]["momentums"]
train_simulation_parameters = aux["train"]["simulation_parameters"]
all_train_metrics = aux["train"]["metrics"]

test_positions = aux["test"]["positions"]
test_momentums = aux["test"]["momentums"]
test_simulation_parameters = aux["test"]["simulation_parameters"]
all_test_metrics = aux["test"]["metrics"]

In [29]:
# Compute actions.
jump = 1
curr_positions, curr_momentums, *_ = train.get_coordinates_for_time_jump(
    train_positions, train_momentums, jump
)

_, _, auxiliary_predictions = state.apply_fn(
    state.params, curr_positions, curr_momentums, 0
)
actions = auxiliary_predictions["actions"]
angles = auxiliary_predictions["current_angles"]

In [32]:
model = pysr.PySRRegressor(niterations=100)

In [46]:
curr_positions, curr_momentums = train.inverse_transform_with_scaler(curr_positions, curr_momentums, scaler)
masses = np.tile(train_simulation_parameters['m'][np.newaxis, :], (curr_positions.shape[0], 1))

(499, 2)

In [47]:
X = np.concatenate([curr_positions, curr_momentums, masses], axis=1)

In [48]:
model.fit(X=X, y=actions)

/Users/ameyad/Documents/google-research/.venv/lib/python3.10/site-packages/pysr/sr.py:1699: UserWarning: The discovered expressions are being reset. Please set `warm_start=True` if you wish to continue to start a search where you left off.
  warnings.warn(
/Users/ameyad/Documents/google-research/.venv/lib/python3.10/site-packages/pysr/sr.py:1225: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Started!

Cycles per second: 2.400e+05
Head worker occupation: 11.3%
Progress: 667 / 3000 total iterations (22.233%)
Best equations for output 1
Hall of Fame:
-----------------------------------------
Complexity  loss       Score     Equation
1           1.375e-06  -1.000e-10  1.4889566
5           1.354e-06  3.927e-03  ((x2 * 1.698073e-5) - -1.4889534)
7           1.017e-06  1.432e-01  (((x3 * x2) * 0.0034503608) - -1.4889301)
9           1.002e-06  7.066e-03  ((((x3 * 0.0034193413) + 1.3871648e-5) * x2) - -1.4889274)
11          1.002e-06  4.831e-05  ((((x3 * 0.0034193413) + 1.3871648e-5) * (x2 + -0.19352844)) - -1.4889274)
13          1.001e-06  8.409e-04  (((((x3 * 0.0034193413) / (x5 + x3)) + 1.3871648e-5) * x2) - -1.4889274)
15          9.762e-07  1.236e-02  (1.5403733 - ((((0.0005618166 / x4) / ((x2 * x3) - 0.49366328)) - -0.010555818) * x4))
17          9.753e-07  4.570e-04  (1.5403733 - ((((0.0005618166 / (x3 + x4)) / ((x2 * x3) - 0.49366328)) - -0.010555818) * x4))
19        

PySRRegressor.equations_ = [
[
	   pick     score                                           equation  \
	0        0.000000                                          1.4889566   
	1        0.003928                 ((x2 * 1.6974162e-5) - -1.4889529)   
	2  >>>>  0.143196            (((x3 * x2) * 0.00345036) - -1.4889303)   
	3        0.007066  ((((x3 * 0.0034186563) + 1.3870111e-5) * x2) -...   
	4        0.100561  (((((x3 * x2) * x2) * 3.601098e-5) * x2) - -1....   
	5        0.008117  ((((((x3 * x2) * x2) + 0.31948254) * 3.601098e...   
	6        0.000388  ((((((x3 * (x2 - 0.3484059)) * x2) + 0.3194825...   
	7        0.015567  (((((x3 * x2) * (x2 + ((x3 / 0.014726842) - 1....   
	8        0.000204  (((((x3 * x2) * ((x2 + ((x3 / 0.014726842) - 1...   
	
	           loss  complexity  
	0  1.375210e-06           1  
	1  1.353770e-06           5  
	2  1.016639e-06           7  
	3  1.002372e-06           9  
	4  8.197528e-07          11  
	5  8.065527e-07          13  
	6  8.059275e-07          15  
	7  7.812218e-07          17  
	8  7.809033e-07          19  
], [
	   pick     score                                           equation  \
	0        0.000000                                         0.54505855   
	1        0.004094                   ((x3 * 0.011272862) + 0.5450467)   
	2  >>>>  0.039467          (((x2 * x0) * 1.3374634e-9) + 0.54505044)   
	3        0.000065   ((((x2 - x3) * x0) * 1.3374634e-9) + 0.54505044)   
	4        0.008750  ((x3 * 0.010755371) + (((x2 * x0) * 1.3374634e...   
	5        0.000005  ((x3 * 0.010755371) + ((((x2 + 0.09094847) * x...   
	6        0.007134  (((x3 + -0.059055537) * (x3 * -0.18079996)) + ...   
	7        0.001270  ((-0.059055537 * (x3 * (-0.18079996 + (x3 * x4...   
	8        0.000079  (((-0.059055537 * 1.0604887) * (x3 * (-0.18079...   
	
	       loss  complexity  
	0  0.000005           1  
	1  0.000005           5  
	2  0.000005           7  
	3  0.000005           9  
	4  0.000005          11  
	5  0.000005          13  
	6  0.000005          15  
	7  0.000005          17  
	8  0.000005          19  
]]